In [2]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-08-08 16:35:17--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.73MB/s    in 0.2s    

2021-08-08 16:35:18 (5.73 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Grocery_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Grocery_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42521656|R26MV8D0KG6QI6|B000SAQCWC|     159713740|The Cravings Plac...|         Grocery|          5|            0|          0|   N|                Y|Using these for y...|As a family aller...| 2015-08-31|
|         US|   12049833|R1OF8GP57AQ1A0|B00509LVIQ|     138680402|Mauna Loa Macadam...|         Grocery|          5|    

In [7]:
#Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20
totalFiltered = df.filter("total_votes>=20")
totalFiltered.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    2622293|R1OAZUG90XPU2U|B00JQ47DO2|     268145085|Epic Bison Bacon ...|         Grocery|          5|          250|        274|   N|                Y|Way better than s...|I love these bars...| 2015-08-31|
|         US|   12244195|R23VPTUJ8I8NR5|B00XWQ2F2K|     138780411|Anna and Sarah Or...|         Grocery|          1|    

In [8]:
#create a new DataFrame to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
helpBytotal = totalFiltered.filter("helpful_votes/total_votes>=0.50")
helpBytotal.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    2622293|R1OAZUG90XPU2U|B00JQ47DO2|     268145085|Epic Bison Bacon ...|         Grocery|          5|          250|        274|   N|                Y|Way better than s...|I love these bars...| 2015-08-31|
|         US|   12244195|R23VPTUJ8I8NR5|B00XWQ2F2K|     138780411|Anna and Sarah Or...|         Grocery|          1|    

In [9]:
#create a new DataFrame that retrieves all the rows where a review was paid
paid = helpBytotal.filter("vine == 'Y'")
paid.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   34773146|R3MKO875WGIEBJ|B00N9IMNKA|      61108636|Vita Coco Organic...|         Grocery|          5|           34|         37|   Y|                N|My Top Coconut Oi...|I'm very strict w...| 2015-08-05|
|         US|   35093059|R3OPNTK61FQ7MP|B00S0AOIFA|     713485594|    V8 Veggie Blends|         Grocery|          5|    

In [11]:
#create a new DataFrame that retrieves all the rows where a review was unpaid
unpaid = helpBytotal.filter("vine == 'N'")
unpaid.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    2622293|R1OAZUG90XPU2U|B00JQ47DO2|     268145085|Epic Bison Bacon ...|         Grocery|          5|          250|        274|   N|                Y|Way better than s...|I love these bars...| 2015-08-31|
|         US|   12244195|R23VPTUJ8I8NR5|B00XWQ2F2K|     138780411|Anna and Sarah Or...|         Grocery|          1|    

In [17]:
#Determine the total number of reviews, the number of 5-star reviews for paid and unpaid
paid_totalNumber = paid.agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "paid_customer_count")
paid_totalNumber.show()

unpaid_totalNumber = unpaid.agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "unpaid_customer_count")
unpaid_totalNumber.show()

paid_fiveStar = paid.filter("star_rating == 5").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "paid_fiveStar_count")
paid_fiveStar.show()

unpaid_fiveStar = unpaid.filter("star_rating == 5").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "unpaid_fiveStar_count")
unpaid_fiveStar.show()

+-------------------+
|paid_customer_count|
+-------------------+
|                 61|
+-------------------+

+---------------------+
|unpaid_customer_count|
+---------------------+
|                28287|
+---------------------+

+-------------------+
|paid_fiveStar_count|
+-------------------+
|                 20|
+-------------------+

+---------------------+
|unpaid_fiveStar_count|
+---------------------+
|                15689|
+---------------------+



In [24]:
#Determine the percentage of 5-star reviews for paid and unpaid.

paid_percentage = paid_fiveStar.withColumn('paid_fiveStar_count',(paid_fiveStar['paid_fiveStar_count']/61)*100).withColumnRenamed("paid_fiveStar_count","paid_fiveStar_percentage")
paid_percentage.show()

unpaid_percentage = unpaid_fiveStar.withColumn('unpaid_fiveStar_count',(unpaid_fiveStar['unpaid_fiveStar_count']/28287)*100).withColumnRenamed("unpaid_fiveStar_count","unpaid_fiveStar_percentage")
unpaid_percentage.show()

+------------------------+
|paid_fiveStar_percentage|
+------------------------+
|       32.78688524590164|
+------------------------+

+--------------------------+
|unpaid_fiveStar_percentage|
+--------------------------+
|        55.463640541591545|
+--------------------------+

